In [191]:
from __future__ import division, print_function
import numpy as np
from hmmlearn.hmm import MultinomialHMM
import poetrytools
import itertools
import time

In [192]:
%run 'preprocessing.ipynb'

In [193]:
def get_rhymes(words):
    num_words = len(words)
    rhymes = np.empty((num_words, num_words))
    
    for i in range(num_words):
        for j in (k for k in range(num_words) if not k == i):
            if poetrytools.rhymes(words[i], words[j]):
                rhymes[i][j] = 1
    return rhymes

In [281]:
quatrain_rhymes = get_rhymes(quatrain_words)
volta_rhymes = get_rhymes(volta_words)
couplet_rhymes = get_rhymes(couplet_words)

In [195]:
reversed_quatrain_model = MultinomialHMM(n_components=30, n_iter=20, verbose=True)
reversed_volta_model = MultinomialHMM(n_components=30, n_iter=20, verbose=True)
reversed_couplet_model = MultinomialHMM(n_components=30, n_iter=20, verbose=True)
reversed_quatrain_model.fit(np.concatenate(reversed_converted_quatrain_lines), quatrain_lengths)
reversed_volta_model.fit(np.concatenate(reversed_converted_volta_lines), volta_lengths)
reversed_couplet_model.fit(np.concatenate(reversed_converted_couplet_lines), couplet_lengths)

         1      -75894.8318             +nan
         2      -62169.0486      +13725.7832
         3      -62161.5162          +7.5324
         4      -62137.7369         +23.7794
         5      -62060.5324         +77.2045
         6      -61822.0680        +238.4644
         7      -61269.6252        +552.4427
         8      -60580.5955        +689.0297
         9      -60150.5293        +430.0662
        10      -59958.2948        +192.2345
        11      -59858.5186         +99.7762
        12      -59779.5475         +78.9710
        13      -59681.8289         +97.7186
        14      -59542.9309        +138.8980
        15      -59362.6478        +180.2831
        16      -59159.5839        +203.0639
        17      -58956.3148        +203.2691
        18      -58763.1790        +193.1358
        19      -58563.6543        +199.5247
        20      -58313.8096        +249.8447
         1      -35845.9006             +nan
         2      -30166.4898       +5679.4108
         3

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=30,
        n_iter=20, params='ste',
        random_state=<mtrand.RandomState object at 0x103563640>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=True)

In [181]:
# checks if line is in iambic pentameter (i.e. 0101010101 stress pattern)
def check_iambic_pentameter(line):
    # get the stresses from cmu dict 
    # if word is 1 syllable, then have the option for it to be stressed or unstressed
    
    stresses = []
    for i in line.split(' '):
        if i != '':
            stress = poetrytools.stress(i)
            if len(stress) == 1:
                stresses.append(['0','1'])
            else:
                stresses.append([stress])
    
    # make combination of all possible stress patterns
    result = [[]]
    final = []
    for pool in stresses:
        result = [x+[y] for x in result for y in pool]
    final = [''.join(i) for i in result]
    
    # return if any pattern fits iambic pentameter 
    return ('0101010101' in final)

def generate_b_line(seed, rhymes, model, length=7):
    b_line = []
    start_probs = model.startprob_
    emission_probs = model.emissionprob_
    transition_probs = model.transmat_

    start_state = np.random.choice(len(start_probs), p = start_probs)
    
    possible_start_emissions = np.where(rhymes[seed] == 1)
    probs = np.array(emission_probs[start_state][possible_start_emissions])

    scaled_probs = probs / sum(probs)
    
    start_emission = np.random.choice(possible_start_emissions[0], p=scaled_probs)
    b_line.append(start_emission)
    
    curr_state = start_state
    for _ in range(length - 1):
        possible_transitions = transition_probs[curr_state]
        curr_state = np.random.choice(len(possible_transitions), p=possible_transitions)
        possible_emissions = emission_probs[curr_state]
        curr_emission = np.random.choice(len(possible_emissions), p=possible_emissions)
        b_line.append(curr_emission)
        
    return b_line

# convert a_line to words
def convert_a_line(sample, words):
    ret = ''
    reversed_list = []
    for word in np.nditer(sample):
        reversed_list.append(word)
    
    i = 0
    for word in reversed(reversed_list):
        curr_word = words[word]
        if i == 0 or len(curr_word) == 1:
            ret += curr_word.title() + ' '
        else:
            ret += curr_word + ' '
        i += 1
    return ret

# convert b_line to words
def convert_b_line(sample, words):
    ret = ''
    i = 0
    for word in reversed(sample):
        curr_word = words[word]
        if i == 0 or len(curr_word) == 1:
            ret += curr_word.title() + ' '
        else:
            ret += curr_word + ' '
        i += 1
    return ret

# generate pair of a_line and b_line 
def generate_pair(model, rhymes, words, length=7):
    while True:
        while True:
            a_line = model.sample(length)[0]
            seed = a_line[0][0]
            a_line = convert_a_line(a_line, words)
            # if not iambic, generate whole line again
            if (check_iambic_pentameter(a_line)):
                break
        if len(np.where(rhymes[seed] == 1)[0]) > 0:
            while True:
                b_line = generate_b_line(seed, rhymes, model)
                b_line = convert_b_line(b_line, words)
                # if not iambic, generate whole line again
                if (check_iambic_pentameter(b_line)):
                    break
            return a_line, b_line
        
# generate 1 sonnet
def generate_rhyming_sonnet():
    start = time.time()
    elapsed = 0
    while elapsed < 2:
        sonnet = ''
        a_lines = []
        b_lines = []

        for _ in range(4):
            a_line, b_line = generate_pair(reversed_quatrain_model, quatrain_rhymes, quatrain_words)
            a_lines.append(a_line)
            b_lines.append(b_line)

        for i in range(2):
            sonnet += a_lines[2 * i] + '\n'
            sonnet += a_lines[2 * i + 1] + '\n'
            sonnet += b_lines[2 * i] + '\n'
            sonnet += b_lines[2 * i + 1] + '\n'

        a_lines = []
        b_lines = []

        for _ in range(2):
            a_line, b_line = generate_pair(reversed_volta_model, volta_rhymes, volta_words)
            a_lines.append(a_line)
            b_lines.append(b_line)

        sonnet += a_lines[0] + '\n'
        sonnet += a_lines[1] + '\n'
        sonnet += b_lines[0] + '\n'
        sonnet += b_lines[1] + '\n'

        a_line, b_line = generate_pair(reversed_couplet_model, couplet_rhymes, couplet_words)
        sonnet += a_line + '\n'
        sonnet += b_line + '\n'
        
        return sonnet
        elapsed = time.time() - start
    return(generate_rhyming_sonnet())
    
# generate 10 sonnets
def generate_10_rhyming_sonnets():
    sonnets = ''
    for i in range(10):
        sonnets += str(i) + '\n' + generate_rhyming_sonnet() + '\n'
    
    f = open("project2data/rhyming_shakespeare2.txt","w")
    f.write(sonnets)
    return sonnets

In [188]:
print(generate_10_rhyming_sonnets())

0
Hast see lord O perfumes addition due 
And graces make gone for antique behind 
Gives painting power earth thou motion hue 
Some own infection your thee summer's brand 
Hath hated why not monsters simple give 
Or tender of dost in eclipse adieu 
When find see doth above abundance live 
From worth looks lust inherit ever due 
Abundant of couldst this as bounty youth 
The sun bud eye before invention lend 
Still my thou and I distillation truth 
On honour she in any sightless fiend 
Remembered love eyes nor herself knows dead 
Shall for give universe in purpose bred 

1
Your which not any outward am anew 
Me thou your proudly hell affections flow 
Describe which princes that all cancelled new 
Beloved giving when A the time slow 
Advantage thy well that self are allayed 
To that your thou beloved of appear 
Alone prescriptions to robs like but aid 
Play many slept doth can appearance dear 
Both wood might fearing physic heaven truth 
Me bearing flourish most of healthful time 
His my n